<a href="https://colab.research.google.com/github/jjikku/OptiGene-Algo/blob/main/ICT_Group_2_ML_AI_Algo_Trading_LSTM_Sep_Predict_test_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Specify the exact path to the CSV file
file_path = "/content/drive/My Drive/NIfty_Dataset/NIFTY_with_Greeks_And_TA_Sep_test_data.csv"

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)
df.columns.tolist()
df_with_timestamp = df.copy()

df.head()

Mounted at /content/drive


,Date/Time,Ticker,Open,High,Low,Close,Volume,Open Interest,INDIAVIX,ATM_OI,...,MACD_15m,RSI_15m,EMA_15m,Stoch_%K_15m,Stoch_%D_15m,ATR_15m,Bollinger_High_15m,Bollinger_Mid_15m,Bollinger_Low_15m,MACD_Hist_15m
0,2021-09-03 09:15:00,NIFTY,17262.5,17296.8,17255.3,17293.3,0,0,14.19,1666100.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,2021-09-03 09:16:00,NIFTY,17292.3,17300.3,17289.0,17289.0,0,0,14.21,1666100.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2,2021-09-03 09:17:00,NIFTY,17288.9,17298.2,17282.0,17297.3,0,0,14.17,2063500.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
3,2021-09-03 09:18:00,NIFTY,17298.8,17312.0,17296.5,17302.3,0,0,14.14,2063500.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
4,2021-09-03 09:19:00,NIFTY,17302.3,17306.7,17301.2,17306.1,0,0,13.78,2063500.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [ ]:
df_to_LSTM  = df.drop(['Ticker'], axis=1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

# List of columns to exclude from normalization (binary target columns + 'Date/Time')
columns_to_exclude = ['Date/Time']

# Get the list of columns to normalize (all columns except the ones to exclude)
columns_to_normalize = [col for col in df_to_LSTM.columns if col not in columns_to_exclude]

# Initialize the feature scaler
feature_scaler = MinMaxScaler()

# Fit and transform the features
df_to_LSTM[columns_to_normalize] = feature_scaler.fit_transform(df_to_LSTM[columns_to_normalize])

# Initialize the target scaler
target_scaler = MinMaxScaler()

# # Assuming your target columns are 'CE_Confidence_Score' and 'PE_Confidence_Score'
# target_columns = ['CE_Confidence_Score', 'PE_Confidence_Score']
# df_to_LSTM[target_columns] = target_scaler.fit_transform(df_to_LSTM[target_columns])


df_to_LSTM['Date/Time'] = df_to_LSTM['Date/Time']



In [ ]:
# # Get the explained variance ratio of the components
# explained_variance_ratio = pca.explained_variance_ratio_
# print(explained_variance_ratio)


In [ ]:
# # Get the PCA components (loadings)
# pca_components = pca.components_
# df_components = pd.DataFrame(pca_components, columns=columns_to_normalize)
# print(df_components)


In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# # Create a DataFrame for the PCA components
# df_components = pd.DataFrame(pca.components_, columns=columns_to_normalize)

# # Plot a heatmap
# plt.figure(figsize=(15, 10))
# sns.heatmap(df_components, cmap='viridis', annot=False)
# plt.title('PCA Component Loadings')
# plt.ylabel('Principal Component')
# plt.xlabel('Feature')
# plt.xticks(rotation=90)  # Rotate feature names for better readability
# plt.show()


In [ ]:
from sklearn.model_selection import train_test_split

# Assuming df_to_LSTM is your DataFrame
# Convert 'Date/Time' to datetime if it's not already
df_to_LSTM['Date/Time'] = pd.to_datetime(df_to_LSTM['Date/Time'])

# Sort the data by 'Date/Time' just in case
df_to_LSTM.sort_values('Date/Time', inplace=True)

# Drop the 'Date/Time' column if it won't be used as a feature
nan_row_indices = df_to_LSTM[df_to_LSTM.isna().any(axis=1)].index
df_to_LSTM.drop(['Date/Time','Volume','Open Interest'], axis=1, inplace=True)

df_to_LSTM.dropna(inplace=True)

# # Split data into features and target
# X = df_to_LSTM.drop(['CE_Confidence_Score','PE_Confidence_Score'], axis=1)
# y = df_to_LSTM[['CE_Confidence_Score','PE_Confidence_Score']]

# # Split into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


In [ ]:
# import numpy as np

# def create_prediction_sequences(input_data, time_steps):
#     Xs = []
#     for i in range(len(input_data) - time_steps):
#         Xs.append(input_data.iloc[i:(i + time_steps)].values)
#     return np.array(Xs)

# # Define the time step
# time_steps = 20

# # Create sequences from your DataFrame for prediction
# X_predict_seq = create_prediction_sequences(df_to_LSTM, time_steps)

import numpy as np

def create_prediction_sequences_rolling_window(input_data, time_steps):
    Xs = []
    # Adjust the loop to allow for the rolling window
    for i in range(len(input_data) - time_steps + 1):
        Xs.append(input_data.iloc[i:(i + time_steps)].values)
    return np.array(Xs)

# Define the time step
time_steps = 20  # Fine-tune this as needed

# Create sequences from your DataFrame for prediction using the rolling window approach
X_predict_seq = create_prediction_sequences_rolling_window(df_to_LSTM, time_steps)


In [ ]:
df_to_LSTM.columns.tolist()

['Open',
 'High',
 'Low',
 'Close',
 'INDIAVIX',
 'ATM_OI',
 'ATM_Call_Volume',
 'ATM_Put_Volume',
 'ATM_Gamma',
 'ATM_Vega',
 'ITM1_OI',
 'ITM1_Call_Volume',
 'ITM1_Put_Volume',
 'ITM1_Gamma',
 'ITM1_Vega',
 'OTM1_OI',
 'OTM1_Call_Volume',
 'OTM1_Put_Volume',
 'OTM1_Gamma',
 'OTM1_Vega',
 'ITM2_OI',
 'ITM2_Call_Volume',
 'ITM2_Put_Volume',
 'ITM2_Gamma',
 'ITM2_Vega',
 'OTM2_OI',
 'OTM2_Call_Volume',
 'OTM2_Put_Volume',
 'OTM2_Gamma',
 'OTM2_Vega',
 'ITM3_OI',
 'ITM3_Call_Volume',
 'ITM3_Put_Volume',
 'ITM3_Gamma',
 'ITM3_Vega',
 'OTM3_OI',
 'OTM3_Call_Volume',
 'OTM3_Put_Volume',
 'OTM3_Gamma',
 'OTM3_Vega',
 'ITM4_OI',
 'ITM4_Call_Volume',
 'ITM4_Put_Volume',
 'ITM4_Gamma',
 'ITM4_Vega',
 'OTM4_OI',
 'OTM4_Call_Volume',
 'OTM4_Put_Volume',
 'OTM4_Gamma',
 'OTM4_Vega',
 'ITM5_OI',
 'ITM5_Call_Volume',
 'ITM5_Put_Volume',
 'ITM5_Gamma',
 'ITM5_Vega',
 'OTM5_OI',
 'OTM5_Call_Volume',
 'OTM5_Put_Volume',
 'OTM5_Gamma',
 'OTM5_Vega',
 'PCR',
 'ATM_PE_OI',
 'ATM_PE_Gamma',
 'ATM_PE_Vega',

In [ ]:
print(df_to_LSTM)

          Open      High       Low     Close  INDIAVIX    ATM_OI  \
20    0.161207  0.155002  0.129907  0.126810  0.389474  0.079580   
21    0.121432  0.112964  0.098832  0.099720  0.363158  0.079580   
22    0.098503  0.119775  0.104907  0.124241  0.368421  0.079580   
23    0.123070  0.126116  0.125234  0.120972  0.368421  0.086087   
24    0.116518  0.115782  0.107944  0.109762  0.384211  0.086087   
...        ...       ...       ...       ...       ...       ...   
3379  0.976369  0.978394  0.979206  0.977814  0.557895  0.496874   
3380  0.983388  0.978394  0.976869  0.975012  0.563158  0.496874   
3381  0.969818  0.967356  0.967523  0.971275  0.563158  0.471924   
3382  0.972391  0.967121  0.963318  0.964970  0.594737  0.471924   
3383  0.962798  0.957492  0.967991  0.962634  0.594737  0.471924   

      ATM_Call_Volume  ATM_Put_Volume  ATM_Gamma  ATM_Vega  ...  MACD_15m  \
20           0.020250        0.033360   0.003570  0.996337  ...  0.426928   
21           0.030079        

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('/content/drive/My Drive/OptiGene Model/my_lstm_Gen5_GA_model')

In [ ]:
predictions = model.predict(X_predict_seq)


100/100 [==============================] - 11s 96ms/step


In [ ]:
# Load the scaler
from joblib import load

target_scaler = load('/content/drive/My Drive/OptiGene Model/target_scaler.joblib')

# Inverse transform your predictions
inverse_predictions = target_scaler.inverse_transform(predictions)

# Convert predictions to a DataFrame
df_predictions = pd.DataFrame({
    'CE_Confidence_Score_Pred': inverse_predictions[:, 0],
    'PE_Confidence_Score_Pred': inverse_predictions[:, 1]
})


In [ ]:
print(df_predictions)

      CE_Confidence_Score_Pred  PE_Confidence_Score_Pred
0                     0.121159                  0.131981
1                     0.121827                  0.132091
2                     0.121723                  0.132205
3                     0.120951                  0.131675
4                     0.120528                  0.131567
...                        ...                       ...
3179                  0.183208                  0.117000
3180                  0.183317                  0.118201
3181                  0.184767                  0.120076
3182                  0.186979                  0.121326
3183                  0.188732                  0.122021

[3184 rows x 2 columns]


In [ ]:
import pandas as pd

# Assuming df_predictions is your DataFrame
# Replace df_predictions with your actual DataFrame variable name

# Find maximum and minimum of CE_Confidence_Score_Pred
max_ce_pred = df_predictions['CE_Confidence_Score_Pred'].max()
min_ce_pred = df_predictions['CE_Confidence_Score_Pred'].min()

# Find maximum and minimum of PE_Confidence_Score_Pred
max_pe_pred = df_predictions['PE_Confidence_Score_Pred'].max()
min_pe_pred = df_predictions['PE_Confidence_Score_Pred'].min()

# Print the results
print("Maximum CE Confidence Score Prediction:", max_ce_pred)
print("Minimum CE Confidence Score Prediction:", min_ce_pred)
print("Maximum PE Confidence Score Prediction:", max_pe_pred)
print("Minimum PE Confidence Score Prediction:", min_pe_pred)


Maximum CE Confidence Score Prediction: 0.23257907
Minimum CE Confidence Score Prediction: -0.07850064
Maximum PE Confidence Score Prediction: 0.34054148
Minimum PE Confidence Score Prediction: -0.09091322


In [ ]:
print(df_predictions)

      CE_Confidence_Score_Pred  PE_Confidence_Score_Pred
0                     0.121159                  0.131981
1                     0.121827                  0.132091
2                     0.121723                  0.132205
3                     0.120951                  0.131675
4                     0.120528                  0.131567
...                        ...                       ...
3179                  0.183208                  0.117000
3180                  0.183317                  0.118201
3181                  0.184767                  0.120076
3182                  0.186979                  0.121326
3183                  0.188732                  0.122021

[3184 rows x 2 columns]


In [ ]:
# Normalize CE Confidence Score to 0-100 range
df_predictions['Normalized_CE_Confidence_Score'] = 100 * (df_predictions['CE_Confidence_Score_Pred'] - min_ce_pred) / (max_ce_pred - min_ce_pred)

# Normalize PE Confidence Score to 0-100 range
df_predictions['Normalized_PE_Confidence_Score'] = 100 * (df_predictions['PE_Confidence_Score_Pred'] - min_pe_pred) / (max_pe_pred - min_pe_pred)

# Check the result
print(df_predictions[['CE_Confidence_Score_Pred', 'PE_Confidence_Score_Pred', 'Normalized_CE_Confidence_Score', 'Normalized_PE_Confidence_Score']])

# Calculate the difference between normalized scores
df_predictions['Confidence_Score_Difference'] = df_predictions['Normalized_CE_Confidence_Score'] - df_predictions['Normalized_PE_Confidence_Score']

# Check the maximum and minimum of the difference
max_difference = df_predictions['Confidence_Score_Difference'].max()
min_difference = df_predictions['Confidence_Score_Difference'].min()

print("Maximum Difference:", max_difference)
print("Minimum Difference:", min_difference)


      CE_Confidence_Score_Pred  PE_Confidence_Score_Pred  \
0                     0.121159                  0.131981   
1                     0.121827                  0.132091   
2                     0.121723                  0.132205   
3                     0.120951                  0.131675   
4                     0.120528                  0.131567   
...                        ...                       ...   
3179                  0.183208                  0.117000   
3180                  0.183317                  0.118201   
3181                  0.184767                  0.120076   
3182                  0.186979                  0.121326   
3183                  0.188732                  0.122021   

      Normalized_CE_Confidence_Score  Normalized_PE_Confidence_Score  
0                          64.182724                       51.661144  
1                          64.397614                       51.686687  
2                          64.364243                       51.7130

In [ ]:
print(df_with_timestamp)

                Date/Time Ticker     Open     High      Low    Close  Volume  \
0     2021-09-03 09:15:00  NIFTY  17262.5  17296.8  17255.3  17293.3       0   
1     2021-09-03 09:16:00  NIFTY  17292.3  17300.3  17289.0  17289.0       0   
2     2021-09-03 09:17:00  NIFTY  17288.9  17298.2  17282.0  17297.3       0   
3     2021-09-03 09:18:00  NIFTY  17298.8  17312.0  17296.5  17302.3       0   
4     2021-09-03 09:19:00  NIFTY  17302.3  17306.7  17301.2  17306.1       0   
...                   ...    ...      ...      ...      ...      ...     ...   
3379  2021-09-16 15:26:00  NIFTY  17632.3  17635.4  17631.3  17633.0       0   
3380  2021-09-16 15:27:00  NIFTY  17635.3  17635.4  17630.3  17631.8       0   
3381  2021-09-16 15:28:00  NIFTY  17629.5  17630.7  17626.3  17630.2       0   
3382  2021-09-16 15:29:00  NIFTY  17630.6  17630.6  17624.5  17627.5       0   
3383  2021-09-16 15:30:00  NIFTY  17626.5  17626.5  17626.5  17626.5       0   

      Open Interest  INDIAVIX     ATM_O

In [ ]:
print(df_to_LSTM)

          Open      High       Low     Close  INDIAVIX    ATM_OI  \
20    0.161207  0.155002  0.129907  0.126810  0.389474  0.079580   
21    0.121432  0.112964  0.098832  0.099720  0.363158  0.079580   
22    0.098503  0.119775  0.104907  0.124241  0.368421  0.079580   
23    0.123070  0.126116  0.125234  0.120972  0.368421  0.086087   
24    0.116518  0.115782  0.107944  0.109762  0.384211  0.086087   
...        ...       ...       ...       ...       ...       ...   
3379  0.976369  0.978394  0.979206  0.977814  0.557895  0.496874   
3380  0.983388  0.978394  0.976869  0.975012  0.563158  0.496874   
3381  0.969818  0.967356  0.967523  0.971275  0.563158  0.471924   
3382  0.972391  0.967121  0.963318  0.964970  0.594737  0.471924   
3383  0.962798  0.957492  0.967991  0.962634  0.594737  0.471924   

      ATM_Call_Volume  ATM_Put_Volume  ATM_Gamma  ATM_Vega  ...  MACD_15m  \
20           0.020250        0.033360   0.003570  0.996337  ...  0.426928   
21           0.030079        

In [ ]:
# Create a new DataFrame from df_to_LSTM without the NaN rows
df_no_nan = df_with_timestamp.loc[~df_with_timestamp.index.isin(nan_row_indices)]

print(df_no_nan)

                Date/Time Ticker     Open     High      Low    Close  Volume  \
20    2021-09-03 09:35:00  NIFTY  17283.9  17284.8  17267.8  17268.6       1   
21    2021-09-03 09:36:00  NIFTY  17266.9  17266.9  17254.5  17257.0       1   
22    2021-09-03 09:37:00  NIFTY  17257.1  17269.8  17257.1  17267.5       1   
23    2021-09-03 09:38:00  NIFTY  17267.6  17272.5  17265.8  17266.1       1   
24    2021-09-03 09:39:00  NIFTY  17264.8  17268.1  17258.4  17261.3       1   
...                   ...    ...      ...      ...      ...      ...     ...   
3379  2021-09-16 15:26:00  NIFTY  17632.3  17635.4  17631.3  17633.0       0   
3380  2021-09-16 15:27:00  NIFTY  17635.3  17635.4  17630.3  17631.8       0   
3381  2021-09-16 15:28:00  NIFTY  17629.5  17630.7  17626.3  17630.2       0   
3382  2021-09-16 15:29:00  NIFTY  17630.6  17630.6  17624.5  17627.5       0   
3383  2021-09-16 15:30:00  NIFTY  17626.5  17626.5  17626.5  17626.5       0   

      Open Interest  INDIAVIX     ATM_O

In [ ]:
# Number of initial rows in df_no_nan without predictions
offset = 19

# Create a DataFrame of NaNs with the same structure as df_predictions
nan_padding = pd.DataFrame(np.nan, index=range(offset), columns=df_predictions.columns)

# Concatenate the NaN DataFrame with df_predictions
df_predictions_padded = pd.concat([nan_padding, df_predictions], ignore_index=True)

# Truncate the padded predictions DataFrame to match the length of df_no_nan
df_predictions_padded = df_predictions_padded.iloc[:len(df_no_nan)]

# Reset indices to ensure alignment
df_no_nan.reset_index(drop=True, inplace=True)
df_predictions_padded.reset_index(drop=True, inplace=True)

# Concatenate the prediction columns with df_no_nan
df_no_nan = pd.concat([df_no_nan, df_predictions_padded], axis=1)
print(df_no_nan)

                Date/Time Ticker     Open     High      Low    Close  Volume  \
0     2021-09-03 09:35:00  NIFTY  17283.9  17284.8  17267.8  17268.6       1   
1     2021-09-03 09:36:00  NIFTY  17266.9  17266.9  17254.5  17257.0       1   
2     2021-09-03 09:37:00  NIFTY  17257.1  17269.8  17257.1  17267.5       1   
3     2021-09-03 09:38:00  NIFTY  17267.6  17272.5  17265.8  17266.1       1   
4     2021-09-03 09:39:00  NIFTY  17264.8  17268.1  17258.4  17261.3       1   
...                   ...    ...      ...      ...      ...      ...     ...   
3198  2021-09-16 15:26:00  NIFTY  17632.3  17635.4  17631.3  17633.0       0   
3199  2021-09-16 15:27:00  NIFTY  17635.3  17635.4  17630.3  17631.8       0   
3200  2021-09-16 15:28:00  NIFTY  17629.5  17630.7  17626.3  17630.2       0   
3201  2021-09-16 15:29:00  NIFTY  17630.6  17630.6  17624.5  17627.5       0   
3202  2021-09-16 15:30:00  NIFTY  17626.5  17626.5  17626.5  17626.5       0   

      Open Interest  INDIAVIX     ATM_O

In [ ]:
# Apply a threshold to convert probabilities to binary predictions
threshold = 40


df_no_nan['PE_Buys_Pred'] = ((df_predictions['Confidence_Score_Difference']) < -1 * threshold).astype(int)

# Filter rows where either CE_Buys_Pred_Class or PE_Buys_Pred_Class is 1
genuine_predictions_pe = df_no_nan[(df_no_nan['PE_Buys_Pred'] == 1)]

# Print the rows with genuine predictions
print(genuine_predictions_pe.count())



Date/Time                         23
Ticker                            23
Open                              23
High                              23
Low                               23
                                  ..
Normalized_CE_Confidence_Score    23
Normalized_PE_Confidence_Score    23
Confidence_Score_Difference       23
PE_Buys_Pred                      23
CE_Buys_Pred                      23
Length: 156, dtype: int64


In [ ]:
# Apply a threshold to convert probabilities to binary predictions
threshold = 40

# df_no_nan['CE_Buys_Pred'] = ((df_no_nan['Confidence_Score_Difference']) > threshold).astype(int)
# df_no_nan['CE_Buys_Pred'] = ((df_no_nan['Normalized_CE_Confidence_Score']) > 95).astype(int)

# Combine the conditions and set 'CE_Buys_Pred'
df_no_nan['CE_Buys_Pred'] = ((df_predictions['Confidence_Score_Difference']) > threshold).astype(int)
# Filter rows where either CE_Buys_Pred_Class or PE_Buys_Pred_Class is 1
genuine_predictions_ce = df_no_nan[(df_no_nan['CE_Buys_Pred'] == 1)]

# Print the rows with genuine predictions
print(genuine_predictions_ce.count())



Date/Time                         47
Ticker                            47
Open                              47
High                              47
Low                               47
                                  ..
Normalized_CE_Confidence_Score    47
Normalized_PE_Confidence_Score    47
Confidence_Score_Difference       47
PE_Buys_Pred                      47
CE_Buys_Pred                      47
Length: 156, dtype: int64


In [ ]:
df_no_nan.head()

,Date/Time,Ticker,Open,High,Low,Close,Volume,Open Interest,INDIAVIX,ATM_OI,...,Bollinger_Mid_15m,Bollinger_Low_15m,MACD_Hist_15m,CE_Confidence_Score_Pred,PE_Confidence_Score_Pred,Normalized_CE_Confidence_Score,Normalized_PE_Confidence_Score,Confidence_Score_Difference,PE_Buys_Pred,CE_Buys_Pred
0,2021-09-03 09:35:00,NIFTY,17283.9,17284.8,17267.8,17268.6,1,0,14.12,1192350.0,...,17284.246667,17269.135899,-0.306044,NaN,NaN,NaN,NaN,NaN,0.0,0.0
1,2021-09-03 09:36:00,NIFTY,17266.9,17266.9,17254.5,17257.0,1,0,14.07,1192350.0,...,17281.453333,17263.073073,-1.936214,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2,2021-09-03 09:37:00,NIFTY,17257.1,17269.8,17257.1,17267.5,1,0,14.08,1192350.0,...,17279.500000,17261.845388,-1.438761,NaN,NaN,NaN,NaN,NaN,0.0,0.0
3,2021-09-03 09:38:00,NIFTY,17267.6,17272.5,17265.8,17266.1,1,0,14.08,1252150.0,...,17278.020000,17259.849812,-1.052912,NaN,NaN,NaN,NaN,NaN,0.0,0.0
4,2021-09-03 09:39:00,NIFTY,17264.8,17268.1,17258.4,17261.3,1,0,14.11,1252150.0,...,17275.986667,17257.614678,-1.074720,NaN,NaN,NaN,NaN,NaN,0.0,0.0


In [ ]:
!pip install plotly

In [ ]:
import plotly.graph_objects as go

# Function to create and customize a candlestick chart
def create_tradingview_like_chart(df):
    # Create a candlestick figure
    fig = go.Figure(data=[go.Candlestick(
        x=df['Date/Time'],
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        increasing_line_color='green',  # Set increasing candle color
        decreasing_line_color='red',    # Set decreasing candle color
        increasing_fillcolor='green',
        decreasing_fillcolor='red'
    )])

    # Plot PE_buys entry points
    fig.add_trace(go.Scatter(
        x=df['Date/Time'][df['PE_Buys_Pred'] == 1],
        y=df['High'][df['PE_Buys_Pred'] == 1] + 2,  # Slightly above the  for visibility
        mode='markers',
        name='PE Buys',
        marker=dict(
            symbol='triangle-down',
            size=10,
            color='rgba(128, 0, 128, 1)',
            line=dict(
                width=2,
                color='rgba(153, 50, 204, 1)'  # Slightly lighter purple for the outline

            )
        )
    ))

    # Plot CE_buys entry points
    fig.add_trace(go.Scatter(
        x=df['Date/Time'][df['CE_Buys_Pred'] == 1],
        y=df['Low'][df['CE_Buys_Pred'] == 1] - 2,
        mode='markers',
        name='CE Buys',
        marker=dict(
            symbol='triangle-up',
            size=10,
            color='rgba(0, 0, 255, 1)',  # Bright blue
            line=dict(
                width=2,
                color='rgba(65, 105, 225, 1)'  # Royal blue for the outline
            )
        )
    ))

    # Customize layout for a more TradingView-like style
    fig.update_layout(
        paper_bgcolor='rgba(18, 18, 18, 1)',  # Dark gray background
        plot_bgcolor='rgba(18, 18, 18, 1)',   # Dark gray plot area
        xaxis_rangeslider_visible=False,
        xaxis=dict(
            gridcolor='rgba(68, 68, 68, 0.5)',  # Lighter gray for gridlines
            showline=True,
            mirror=True,
            linecolor='rgba(68, 68, 68, 1)',  # Lighter gray for axis lines
        ),
        yaxis=dict(
            gridcolor='rgba(68, 68, 68, 0.5)',
            showline=True,
            mirror=True,
            linecolor='rgba(68, 68, 68, 1)',
        ),
        font=dict(
            family="Verdana, sans-serif",
            size=12,
            color="white"  # White font color for visibility on a dark background
        )
    )


    # Add range selector buttons
    fig.update_xaxes(
        rangeselector=dict(
            buttons=list([
                {'count': 3, 'label': '3m', 'step': 'minute', 'stepmode': 'backward'},
                {'count': 5, 'label': '5m', 'step': 'minute', 'stepmode': 'backward'},
                {'count': 10, 'label': '10m', 'step': 'minute', 'stepmode': 'backward'},
                {'count': 15, 'label': '15m', 'step': 'minute', 'stepmode': 'backward'},
                {'count': 1, 'label': '1M', 'step': 'month', 'stepmode': 'backward'},
                {'count': 6, 'label': '6M', 'step': 'month', 'stepmode': 'backward'},
                {'step': 'all'}
            ]),
            bgcolor="rgba(0,0,0,0.1)"
        )
    )

    # Show the figure
    fig.show()

# Create and show the chart
create_tradingview_like_chart(df_no_nan)


In [67]:
import numpy as np

def calculate_profit_loss_and_stats(df, threshold, stop_loss_margin):
    profit_loss = 0
    in_trade = False
    trade_type = None
    entry_price = 0
    trade_returns = []  # List to store returns of each trade
    num_trades = 0
    num_profit_trades = 0
    num_loss_trades = 0

    for i in range(2, len(df)):
        if in_trade:
            if trade_type == 'CE' and df['High'][i] >= entry_price + stop_loss_margin:
                trade_result = entry_price - df['Low'][i]
                profit_loss += trade_result
                trade_returns.append(trade_result)
                in_trade = False
            elif trade_type == 'PE' and df['Low'][i] <= entry_price - stop_loss_margin:
                trade_result = df['High'][i] - entry_price
                profit_loss += trade_result
                trade_returns.append(trade_result)
                in_trade = False
            elif i + 5 < len(df) and not df[trade_type + '_Buys_Pred'][i:i+5].any():
                exit_price = df['Close'][i + 4]
                if trade_type == 'CE':
                    trade_result = exit_price - entry_price
                else:
                    trade_result = entry_price - exit_price
                profit_loss += trade_result
                trade_returns.append(trade_result)
                in_trade = False
                num_trades += 1
                if trade_result > 0:
                    num_profit_trades += 1
                else:
                    num_loss_trades += 1
        else:
            if df['CE_Buys_Pred'][i-2:i+1].all():
                in_trade = True
                trade_type = 'CE'
                entry_price = df['Close'][i]
            elif df['PE_Buys_Pred'][i-2:i+1].all():
                in_trade = True
                trade_type = 'PE'
                entry_price = df['Close'][i]

    # Calculating Sharpe Ratio
    trade_returns = np.array(trade_returns) * 0.7 * 1000  # Adjusting trade returns based on delta and quantity
    daily_returns = trade_returns / 200000  # Assuming 200000 as initial capital

    # Annualizing returns and standard deviation
    annualized_return = np.mean(daily_returns) * (252 / 9)  # Adjust for 9 trading days to annualize
    annualized_std = np.std(daily_returns) * np.sqrt(252 / 9)

    # Assuming risk-free rate is 0 for simplification
    sharpe_ratio = annualized_return / annualized_std if annualized_std != 0 else 0

    stats = {
        'Period': '9 Days',
        'Starting Capital' : 200000,
        'Total Profit/Loss': profit_loss * 0.7 * 1000,
        'Number of Trades': num_trades,
        'Profitable Trades': num_profit_trades,
        'Loss-making Trades': num_loss_trades,
        'Sharpe Ratio': sharpe_ratio
    }

    return stats

# Apply the function to your DataFrame
stats = calculate_profit_loss_and_stats(df_no_nan, threshold, 10)
for key, value in stats.items():
    print(f"{key}: {value}")



Period: 9 Days
Starting Capital: 200000
Total Profit/Loss: 42070.00000000153
Number of Trades: 14
Profitable Trades: 10
Loss-making Trades: 4
Sharpe Ratio: 2.697948087529585


In [68]:
import numpy as np
import pandas as pd

def calculate_profit_loss_and_stats(df, threshold, stop_loss_margin):
    profit_loss = 0
    in_trade = False
    trade_type = None
    entry_price = 0
    trade_returns = []  # List to store returns of each trade
    num_trades = 0
    num_profit_trades = 0
    num_loss_trades = 0
    trade_details = []  # List to store details of each trade

    for i in range(2, len(df)):
        if in_trade:
            # Handling exit conditions
            exit_condition_met = False
            trade_result = 0
            exit_price = 0
            if trade_type == 'CE' and df['High'][i] >= entry_price + stop_loss_margin:
                exit_price = df['Low'][i]
                trade_result = entry_price - exit_price
                exit_condition_met = True
            elif trade_type == 'PE' and df['Low'][i] <= entry_price - stop_loss_margin:
                exit_price = df['High'][i]
                trade_result = exit_price - entry_price
                exit_condition_met = True
            elif i + 5 < len(df) and not df[trade_type + '_Buys_Pred'][i:i+5].any():
                exit_price = df['Close'][i + 4]
                trade_result = (exit_price - entry_price) if trade_type == 'CE' else (entry_price - exit_price)
                exit_condition_met = True

            if exit_condition_met:
                profit_loss += trade_result
                trade_returns.append(trade_result)
                in_trade = False
                num_trades += 1
                if trade_result > 0:
                    num_profit_trades += 1
                else:
                    num_loss_trades += 1
                trade_details.append({
                    'Date/Time': df.index[i],
                    'Trade Type': trade_type,
                    'Entry Price': entry_price,
                    'Exit Price': exit_price,
                    'Profit/Loss Points': trade_result * 0.7 * 1000
                })

        else:
            # Handling entry conditions
            if df['CE_Buys_Pred'][i-2:i+1].all():
                in_trade = True
                trade_type = 'CE'
                entry_price = df['Close'][i]
            elif df['PE_Buys_Pred'][i-2:i+1].all():
                in_trade = True
                trade_type = 'PE'
                entry_price = df['Close'][i]

    # Calculate Sharpe Ratio
    trade_returns = np.array(trade_returns) * 0.7 * 1000  # Adjusting trade returns
    daily_returns = trade_returns / 200000  # Capital assumption
    annualized_return = np.mean(daily_returns) * (252 / 9)  # Adjust for 9 trading days
    annualized_std = np.std(daily_returns) * np.sqrt(252 / 9)
    sharpe_ratio = annualized_return / annualized_std if annualized_std != 0 else 0

    stats = {
        'Period': '9 Days',
        'Starting Capital': 200000,
        'Total Profit/Loss': profit_loss * 0.7 * 1000,
        'Number of Trades': num_trades,
        'Profitable Trades': num_profit_trades,
        'Loss-making Trades': num_loss_trades,
        'Sharpe Ratio': sharpe_ratio,
        'Trade Details': pd.DataFrame(trade_details)
    }

    return stats

# Apply the function to your DataFrame
stats = calculate_profit_loss_and_stats(df_no_nan, threshold, 10)
for key, value in stats.items():
    if key != 'Trade Details':
        print(f"{key}: {value}")

# Display trade details
print("\nTrade Details:")
print(stats['Trade Details'])


Period: 9 Days
Starting Capital: 200000
Total Profit/Loss: 42070.00000000153
Number of Trades: 19
Profitable Trades: 11
Loss-making Trades: 8
Sharpe Ratio: 2.697948087529585

Trade Details:
    Date/Time Trade Type  Entry Price  Exit Price  Profit/Loss Points
0         736         PE      17357.8     17352.7             -3570.0
1        1237         PE      17331.2     17335.3              2870.0
2        1614         CE      17308.7     17309.0              -210.0
3        1620         CE      17313.3     17319.3             -4200.0
4        1625         CE      17323.0     17326.3             -2310.0
5        1646         CE      17318.7     17331.7              9100.0
6        1658         CE      17336.5     17334.3             -1540.0
7        2089         PE      17347.4     17339.7              5390.0
8        2114         PE      17363.0     17357.7              3710.0
9        2551         CE      17455.1     17461.7              4620.0
10       2627         CE      17459.2   